In [20]:
import sys, os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir) + "/src/"
sys.path.insert(0, parentdir) 

from modeling import filter_depth, filter_rank, filter_value, value_path, value_EV, value_max, value_sum, value_levelmean
from modelchecking import trialwise_rewards, trialwise_greedydiff, trialwise_chooseleft
from analysis import Analyzer, lmm, glmm
from plots import set_helvetica_style
from utils import colormaps, report_p_value, strsimplify, get_stochasticity_levels, alphabet, format_games
from modeling import preprocess_data, make_pov_array, Model, MultiStart
import json

%load_ext autoreload
%autoreload 2

def get_colormap(type_):
    return {"R": colormaps["arctic"], "T": colormaps["berry"], "V": colormaps["grass"]}[type_]

def get_filter_and_value_functions(type_):
    # which filter functions to compare to
    compare_filter_fns = [
        ["depth", filter_depth],
        ["rank", filter_rank],
        ["value", filter_value]]

    compare_value_fns = [
        ["path", value_path], 
        ["max", value_max],
        ["sum", value_sum], 
        ["level-mean", value_levelmean]]

    if type_ == "R" or type_ == "T": 
        compare_value_fns.append(["EV", value_EV])
    return compare_filter_fns, compare_value_fns

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
type_ = "R"
datafile = f"../data/raw/data_{type_}.json"
print("data file\t", datafile)
with open(datafile) as f:
    data = json.load(f)

data file	 ../data/raw/data_R.json


In [22]:
user = list(data.keys())[0]

sim_user = f"sim_{user}"
model = Model(filter_depth, value_path, variant=type_)

default_params = {"model": model.name, "filter_params": {p: np.random.choice(8) for p in get_stochasticity_levels(type_)}}
flexible_params = {"lapse": np.random.uniform(0, 1), "inv_temp": np.random.uniform(-4, 4)}
sim_params = {**default_params, **flexible_params}

games = format_games(data[user]["data"])
simulated_data = []
n_repeats = 20

for _ in tqdm.tqdm(range(n_repeats), total=n_repeats, desc=f"simulating {user}"): 
    prediction = model.predict(sim_params, games)
    simulated_data.extend([
            {
                "name": f"game_{user}_c{condition}_g{game}",
                "p": get_stochasticity_levels(type_)[condition],
                "boards": prediction.boards.isel(conditions=condition, games=game).values,
                "oracle": prediction.oracles.isel(conditions=condition, games=game, trials=0).values,
                "tuplepath": prediction.paths.isel(conditions=condition, games=game).values,
                "path": [f'{a},{b}' for a, b in prediction.paths.isel(conditions=condition, games=game).values],
                "actions": prediction.choose_left.isel(conditions=condition, games=game).astype(bool).values,
                "is_transition": prediction.is_transition.isel(conditions=condition, games=game).astype(bool).values,
                "trials": [{"rt": 0} for _ in range(7)],
            }
            for condition in range(5) for game in range(5)])

simulated_games = format_games(simulated_data)
multistart = MultiStart(model, simulated_games, {k:5 for k in flexible_params.keys()}, use_grid = False, n=1)

simulating user0: 100%|██████████| 20/20 [00:00<00:00, 22.21it/s]


In [26]:
param_specs = [
    {
        "name": "main", 
        "params": {"inv_temp": 5, "lapse": 5},
        "conditional_filter_params": True,
        "multistart_n": 100,
    },
    {
        "name": "fixed_depth_variable_beta",
        "params": {"lapse": 5, **{f"condition_inv_temp_{i}": 5 for i in range(5)}},
        "conditional_filter_params": False,
        "multistart_n": 100,
    }, 
    {
        "name": "variable_depth_variable_lapse",
        "params": {"inv_temp": 5, **{f"condition_lapse_{i}": 5 for i in range(5)}},
        "conditional_filter_params": True,
        "multistart_n": 100,
    },
]

main
['inv_temp', 'lapse']
fixed_depth_variable_beta
['lapse', 'condition_inv_temp_0', 'condition_inv_temp_1', 'condition_inv_temp_2', 'condition_inv_temp_3', 'condition_inv_temp_4']
variable_depth_variable_lapse
['inv_temp', 'condition_lapse_0', 'condition_lapse_1', 'condition_lapse_2', 'condition_lapse_3', 'condition_lapse_4']


In [31]:
from utils import NpEncoder, format_games, get_stochasticity_levels

users = [list(data.keys())[0]]
for param_spec in param_specs:
    name = param_spec["name"]
    named_params = list(param_spec["params"].keys())

    for user in users:
        sim_user = f"sim_{user}"
        model = Model(filter_depth, value_path, variant=type_)

        sim_params = {"model": model.name, "filter_params": {p: np.random.choice(8) for p in get_stochasticity_levels(type_)}}
        for param in named_params:
            lo, hi = model.default_bounds[param]
            sim_params[param] = np.random.uniform(lo, hi)

        games = format_games(data[user]["data"])
        simulated_data = []

        for _ in tqdm.tqdm(range(n_repeats), total=n_repeats, desc=f"simulating {user}"): 
            prediction = model.predict(sim_params, games)
            simulated_data.extend([
                    {
                        "name": f"game_{user}_c{condition}_g{game}",
                        "p": get_stochasticity_levels(type_)[condition],
                        "boards": prediction.boards.isel(conditions=condition, games=game).values,
                        "oracle": prediction.oracles.isel(conditions=condition, games=game, trials=0).values,
                        "tuplepath": prediction.paths.isel(conditions=condition, games=game).values,
                        "path": [f'{a},{b}' for a, b in prediction.paths.isel(conditions=condition, games=game).values],
                        "actions": prediction.choose_left.isel(conditions=condition, games=game).astype(bool).values,
                        "is_transition": prediction.is_transition.isel(conditions=condition, games=game).astype(bool).values,
                        "trials": [{"rt": 0} for _ in range(7)],
                    }
                    for condition in range(5) for game in range(5)
            ])

        simulated_games = format_games(simulated_data)
        multistart = MultiStart(model, simulated_games, {k:5 for k in named_params}, use_grid = False, n=1)
        multistart.sweep()

        filedir = f"../data/sim/{name}/{type_}_{model.name}"
        os.makedirs(filedir, exist_ok=True)

        with open(f"{filedir}/{sim_user}_sim.json", "w") as f:
            json.dump(sim_params, f, cls=NpEncoder)

        with open(f"{filedir}/{sim_user}_fit.json", "w") as f:
            json.dump(multistart.best, f, cls=NpEncoder)

  0%|          | 0/1 [00:00<?, ?it/s]/Users/jordanlei/miniconda3/lib/python3.13/site-packages/xarray/computation/apply_ufunc.py:821: RuntimeWarning: divide by zero encountered in log
  result_data = func(*input_data)
/Users/jordanlei/miniconda3/lib/python3.13/site-packages/xarray/computation/apply_ufunc.py:821: RuntimeWarning: divide by zero encountered in log
  result_data = func(*input_data)
simulating user0:   0%|          | 0/20 [00:00<?, ?it/s]/Users/jordanlei/miniconda3/lib/python3.13/site-packages/xarray/computation/apply_ufunc.py:821: RuntimeWarning: overflow encountered in exp
  result_data = func(*input_data)
/Users/jordanlei/miniconda3/lib/python3.13/site-packages/xarray/computation/apply_ufunc.py:821: RuntimeWarning: overflow encountered in exp
  result_data = func(*input_data)
/Users/jordanlei/miniconda3/lib/python3.13/site-packages/xarray/computation/apply_ufunc.py:821: RuntimeWarning: overflow encountered in exp
  result_data = func(*input_data)
simulating user0:  15%|█▌

{'lapse': 5, 'inv_temp': 5}

In [ ]:
import tqdm

    ])

simulating user0: 100%|██████████| 20/20 [00:01<00:00, 14.80it/s]


  0%|          | 0/100 [00:00<?, ?it/s]

/Users/jordanlei/miniconda3/lib/python3.13/site-packages/xarray/computation/apply_ufunc.py:821: RuntimeWarning: overflow encountered in exp
  result_data = func(*input_data)
/Users/jordanlei/miniconda3/lib/python3.13/site-packages/xarray/computation/apply_ufunc.py:821: RuntimeWarning: divide by zero encountered in log
  result_data = func(*input_data)
/Users/jordanlei/miniconda3/lib/python3.13/site-packages/xarray/computation/apply_ufunc.py:821: RuntimeWarning: overflow encountered in exp
  result_data = func(*input_data)
/Users/jordanlei/miniconda3/lib/python3.13/site-packages/xarray/computation/apply_ufunc.py:821: RuntimeWarning: overflow encountered in exp
  result_data = func(*input_data)
/Users/jordanlei/miniconda3/lib/python3.13/site-packages/xarray/computation/apply_ufunc.py:821: RuntimeWarning: divide by zero encountered in log
  result_data = func(*input_data)
/Users/jordanlei/miniconda3/lib/python3.13/site-packages/xarray/computation/apply_ufunc.py:821: RuntimeWarning: divide

KeyboardInterrupt: 